In [22]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

2023-10-10 11:10:35.932807: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 11:10:40.630236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
from transformers import LlamaModel, LlamaConfig, LlamaForCausalLM

# Initializing a LLaMA llama-7b style configuration
configuration = LlamaConfig()

# Initializing a model from the llama-7b style configuration
model = LlamaForCausalLM(configuration)

# Accessing the model configuration
configuration = model.config

In [24]:
import pandas as pd 
test_data = pd.read_csv('/workspaces/Wikistim-Summarization/00_source_data/uncleaned_text.csv')

In [25]:
my_test = test_data['full_article_text'][1]

In [37]:
my_test

'RESEARCH ARTICLE A multicenter real-world review of 10 kHz SCS outcomes for treatment of chronic trunk and/or limb pain Thomas Stauss1, Faycal El Majdoub2 , Dawood Sayed3, Gernot Surges4, William S. Rosenberg5, Leonardo Kapural6, Richard Bundschu7, Abdul Lalkhen8, Nileshkumar Patel1, Bradford Gliner9, Jeyakumar Subbaroyan9, Anand Rotte9, Deborah R. Edgar10, Martin Bettag4 & Mohammad Maarouf2 1Advanced Pain Management, Greenﬁeld, Wisconsin 2Department of Stereotactic and Functional Neurosurgery, Cologne Merheim Medical Center, University of Witten/Herdecke, Cologne, Germany 3Department of Anesthesiology and Pain Medicine, University of Kansas Medical Center, Kansas City, Kansas 4KH Barmherzige Br€uder, Trier, Germany 5Center for the Relief of Pain, Kansas City, Missouri 6Carolinas Pain Institute, Winston-Salem, North Carolina 7Coastal Orthopedics and Pain Medicine, Bradenton, Florida 8The Manchester and Salford Pain Centre, Salford, United Kingdom 9Nevro Corp, Redwood City, California 

In [38]:
test_data['Downloaded PDF Path'][1]

'/workspaces/Wikistim-Summarization/00_source_data/downloaded_pdfs/pdf_1.pdf'

In [36]:
test_data.head(2)

,PUBLICATION INFORMATION,Author(s),Title,Journal,"Volume, issue, pages",Year,Full text link,PUBMED link,STUDY DESCRIPTION AND METHODS,Study design,...,Facility type,DATA EXTRACTION,Awaiting completion. You may use the above link to download a CSV,Checked out by/date due,Submitted by/date submitted,Submission reviewed by/date reviewed,Unnamed: 214,Unnamed: 215,Downloaded PDF Path,full_article_text
0,,"Al-Kaisy A, Van Buyten JP, Carganillo R, Caraw...","10 kHz SCS therapy for chronic pain, effects o...",Sci Rep,9(1):11441,2019,https://www.nature.com/articles/s41598-019-477...,https://www.ncbi.nlm.nih.gov/pubmed/31391503,,Post hoc analysis of data from two published s...,...,NaN,,NaN,NaN,Anand Rotte/3 Sept 2019,JBS/finalized May 2020,NaN,NaN,/workspaces/Wikistim-Summarization/00_source_d...,1 Scientific RepoRtS | (2019) 9:11441 ...
1,,"Stauss T, El Majdoub F, Sayed D, Surges G, Ros...",A multicenter real-world review of 10 kHz SCS ...,Ann Clin Transl Neurol,6(3):496-507,2019,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pubmed/30911573,,"Multicenter, retrospective analysis with no co...",...,NaN,,NaN,NaN,"Anand Rotte, M Pharm, PhD",Jane Shipley July 2019,NaN,NaN,/workspaces/Wikistim-Summarization/00_source_d...,RESEARCH ARTICLE A multicenter real-world revi...


In [26]:
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
my_encoded_text = tokenizer.encode(my_test)

Token indices sequence length is longer than the specified maximum sequence length for this model (13571 > 2048). Running this sequence through the model will result in indexing errors


In [27]:
len(my_encoded_text)

13571

In [54]:
#chunk my_encoded_text into 512 token chunks
my_encoded_text = tokenizer.encode(my_test)

#chunk my_encoded_text into 512 token chunks
chunk_size = 450

my_encoded_text_chunks = [my_encoded_text[i:i+chunk_size] for i in range(0, len(my_encoded_text), chunk_size)]

len(my_encoded_text_chunks)


25

In [29]:
my_encoded_text_chunks[0]

[1,
 5195,
 1660,
 1718,
 3210,
 9033,
 29911,
 2965,
 1307,
 319,
 1773,
 293,
 5893,
 1855,
 29899,
 11526,
 9076,
 310,
 29871,
 29896,
 29900,
 413,
 12661,
 317,
 9295,
 714,
 26807,
 363,
 14502,
 310,
 17168,
 293,
 534,
 2960,
 322,
 29914,
 272,
 2485,
 29890,
 6788,
 5569,
 624,
 11214,
 29896,
 29892,
 383,
 388,
 1052,
 1260,
 6973,
 29881,
 283,
 29890,
 29906,
 1919,
 7266,
 6115,
 14891,
 287,
 29941,
 29892,
 402,
 824,
 327,
 317,
 2007,
 267,
 29946,
 29892,
 4667,
 317,
 29889,
 5678,
 21845,
 29945,
 29892,
 10255,
 6491,
 9622,
 3631,
 29953,
 29892,
 6123,
 24240,
 816,
 29884,
 29955,
 29892,
 17860,
 352,
 365,
 2235,
 3169,
 29947,
 29892,
 405,
 5475,
 29882,
 29895,
 24540,
 4121,
 295,
 29896,
 29892,
 15862,
 4006,
 402,
 1915,
 261,
 29929,
 29892,
 435,
 1032,
 557,
 24540,
 3323,
 1646,
 29877,
 10094,
 29929,
 29892,
 530,
 392,
 9664,
 371,
 29929,
 29892,
 7089,
 272,
 801,
 390,
 29889,
 2155,
 5397,
 29896,
 29900,
 29892,
 6502,
 350,
 1803,
 351,


In [41]:
#unencode the first chunk
tokenizer.decode(my_encoded_text_chunks[1])


'ives: High-frequency spinal cord stimulation (HF-SCS) at 10 kHz has proven to be efﬁcacious in the treatment of chronic back and leg pain in a randomized, controlled, trial (SENZA-RCT). However, large observational studies have yet to be published. Therefore, we performed a real-world, mul- ticenter, retrospective, review of therapy efﬁcacy in 1660 patients with chronic trunk and/or limb pain. Methods: Data were collected in a real- world environment and retrospectively sourced from a global database. Included patients were trialed and/or permanently implanted with HF-SCS at 10 kHz between April 2014 and January 2018. We evaluated responder rates at 3, 6, and 12 months post-implantation. Response was deﬁned as ≥50% pain relief from baseline. A last visit analysis included responder rate along with overall change in function, sleep, quality of life, and medication intake versus baseline. Results: Eighty-four percent of our HF-SCS-treated patients had both chronic back and leg pain. At 

In [43]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="lxyuan/distilbart-finetuned-summarization", max_length=10, min_length=1)


topic = "Find the number of participants in the study:"
text = f"{topic}: {tokenizer.decode(my_encoded_text_chunks[1])}"

pipe(text)


[{'generated_text': 'high-frequency spinal cord stimulation'}]

In [51]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model="lxyuan/distilbart-finetuned-summarization")

# Define the text to summarize and the topic to focus on
topic = "Number of participants in the study:"
text = "ives: High-frequency spinal cord stimulation (HF-SCS) at 10 kHz has proven to be efﬁcacious in the treatment of chronic back and leg pain in a randomized, controlled, trial (SENZA-RCT). However, large observational studies have yet to be published. Therefore, we performed a real-world, mul- ticenter, retrospective, review of therapy efﬁcacy in 1660 patients with chronic trunk and/or limb pain. Methods: Data were collected in a real- world environment and retrospectively sourced from a global database. Included patients were trialed and/or permanently implanted with HF-SCS at 10 kHz between April 2014 and January 2018. We evaluated responder rates at 3, 6, and 12 months post-implantation. Response was deﬁned as ≥50% pain relief from baseline. A last visit analysis included responder rate along with overall change in function, sleep, quality of life, and medication intake versus baseline. Results: Eighty-four percent of our HF-SCS-treated patients had both chronic back and leg pain. At least 70% of patients reported response to therapy throughout 12 months of follow-up. This sustained responder rate was corroborated by the last visit value (74.1%). Most patients reported concomitant improvements in function (72.3%), sleep (68.0%), and quality of life (90.3%) at their last visit versus baseline. Thirty- two percent of patients reported decreased medication intake at their last visit. Interpretation: Sustained and effective pain relief was experienced by >70% of our HF-SCS-treated patients, consistent with the ﬁndings of a pre- viously published randomized, controlled, trial. Our review provides comple- mentary evidence to support the treatment of chronic back and leg pain with this therapy. Introduction Chronic pain is a pervasive health issue worldwide.1,2 It places a substantial burden on society, families, and indi- viduals.3 Low back pain is one of the most prevalent chron"

# Generate the summary with the topic as a prefix
prefix = f"{topic}: "
summary = summarizer(prefix + text, max_length=29, min_length=10, do_sample=False)

# Print the summary
print(summary[0])

{'summary_text': 'high-frequency spinal cord stimulation (HF-SCS) at 10 kHz has proven to be efﬁcacious'}


In [50]:
tokenizer.decode(my_encoded_text_chunks[1])

{'ives: High-frequency spinal cord stimulation (HF-SCS) at 10 kHz has proven to be efﬁcacious in the treatment of chronic back and leg pain in a randomized, controlled, trial (SENZA-RCT). However, large observational studies have yet to be published. Therefore, we performed a real-world, mul- ticenter, retrospective, review of therapy efﬁcacy in 1660 patients with chronic trunk and/or limb pain. Methods: Data were collected in a real- world environment and retrospectively sourced from a global database. Included patients were trialed and/or permanently implanted with HF-SCS at 10 kHz between April 2014 and January 2018. We evaluated responder rates at 3, 6, and 12 months post-implantation. Response was deﬁned as ≥50% pain relief from baseline. A last visit analysis included responder rate along with overall change in function, sleep, quality of life, and medication intake versus baseline. Results: Eighty-four percent of our HF-SCS-treated patients had both chronic back and leg pain. At

In [21]:
for i in range(0, len(my_encoded_text_chunks)):
    
    text = tokenizer.decode(my_encoded_text_chunks[i])
    pipe(text)
    print('chunk', i, 'of', len(my_encoded_text_chunks), 'complete')

NameError: name 'my_encoded_text_chunks' is not defined

In [19]:
%pip install -q git+https://github.com/huggingface/transformers.git

Note: you may need to restart the kernel to use updated packages.


In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

encoder_input_str = "translate English to German: How old are you?"

input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids

outputs = model.generate(
    input_ids,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
     

/home/codespace/.python/current/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


/home/codespace/.python/current/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Output:
----------------------------------------------------------------------------------------------------
Wie alt bist du?


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = "google/flan-t5-xxl"
model_dir = "./saved_model"  # Choose a directory for saving the model

# Try to load the model from the cache or from the specified directory
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
except OSError:
    # If the model is not found in cache or directory, download and save it
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    model.save_pretrained(model_dir)


In [52]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer


model_name = "google/flan-t5-xxl"  # Example model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-xxl and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
text = "Find the number of participants in the study:: ives: High-frequency spinal cord stimulation (HF-SCS) at 10 kHz has proven to be efﬁcacious in the treatment of chronic back and leg pain in a randomized, controlled, trial (SENZA-RCT). However, large observational studies have yet to be published. Therefore, we performed a real-world, mul- ticenter, retrospective, review of therapy efﬁcacy in 1660 patients with chronic trunk and/or limb pain. Methods: Data were collected in a real- world environment and retrospectively sourced from a global database. Included patients were trialed and/or permanently implanted with HF-SCS at 10 kHz between April 2014 and January 2018. We evaluated responder rates at 3, 6, and 12 months post-implantation. Response was deﬁned as ≥50% pain relief from baseline. A last visit analysis included responder rate along with overall change in function, sleep, quality of life, and medication intake versus baseline. Results: Eighty-four percent of our HF-SCS-treated patients had both chronic back and leg pain. At least 70% of patients reported response to therapy throughout 12 months of follow-up. This sustained responder rate was corroborated by the last visit value (74.1%). Most patients reported concomitant improvements in function (72.3%), sleep (68.0%), and quality of life (90.3%) at their last visit versus baseline. Thirty- two percent of patients reported decreased medication intake at their last visit. Interpretation: Sustained and effective pain relief was experienced by >70% of our HF-SCS-treated patients, consistent with the ﬁndings of a pre- viously published randomized, controlled, trial. Our review provides comple- mentary evidence to support the treatment of chronic back and leg pain with this therapy. Introduction Chronic pain is a pervasive health issue worldwide.1,2 It places a s"

inputs = tokenizer(text, return_tensors="pt")

In [60]:
outputs = model(**inputs)
start_scores, end_scores = outputs.start_logits, outputs.end_logits


answer_start = start_scores.argmax()
answer_end = end_scores.argmax() + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
print("Answer:", answer)

Answer: ives: High-frequency spinal cord stimulation (HF-SCS) at 10 kHz has proven to be efficacious in the treatment of chronic back and leg pain in a randomized, controlled, trial (SENZA-RCT). However, large observational studies have yet to be published. Therefore, we performed a real-world, mul- ticenter, retrospective, review of therapy efficacy in 1660 patients with chronic trunk and/or limb pain. Methods: Data were collected in a real- world environment and retrospectively sourced from a global database. Included patients were trialed and/or permanently implanted with HF-SCS at 10 kHz between April 2014 and January 2018. We evaluated responder rates at 3, 6, and 12 months post-implantation. Response was defined as <unk>50% pain relief from baseline. A last visit analysis included responder rate along with overall change in function, sleep, quality of life, and medication intake versus baseline. Results: Eighty-four percent of our HF-SCS-treated patients had both chronic back and

In [64]:
from transformers import pipeline
from random import randrange        

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", tokenizer=tokenizer, model="philschmid/flan-t5-base-samsum", device=0)



# summarize dialogue
res = summarizer(text)

print(f"flan-t5-base summary:\n{res[0]['summary_text']}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 15.77 GiB total capacity; 15.27 GiB already allocated; 87.12 MiB free; 15.27 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF